# CRS Canada immigration metrics analysis
We investigate the Comprehensive Ranking Systme (CRS) metrics that canada uses to grant permanent residence invitations and project predicted cutoff values for the incoming cohort.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_html('https://www.canada.ca/en/immigration-refugees-citizenship/corporate/mandate/policies-operational-instructions-agreements/ministerial-instructions/express-entry-rounds.html')
df = data[0]
df.head(4)